In [1]:
import os
# Check if we are in the notebooks folder, if so, move up
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\PC\Regression_ML_EndtoEnd


In [2]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [3]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\PC\Regression_ML_EndtoEnd\.venv\Scripts\python.exe
3.0.4
c:\Users\PC\Regression_ML_EndtoEnd\.venv\Lib\site-packages\xgboost\__init__.py


In [4]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\PC\Regression_ML_EndtoEnd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [6]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
import os

# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================

# FIX: Use a relative path so it works on ANY computer
# This creates the 'mlruns' folder right inside your project
mlflow.set_tracking_uri("file:./mlruns")

mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/12/09 11:24:52 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-09 11:24:52,370] A new study created in memory with name: no-name-ba66bd4d-ef16-4009-b47d-7fb9c6b72515
[I 2025-12-09 11:25:44,880] Trial 0 finished with value: 70210.4968106389 and parameters: {'n_estimators': 568, 'max_depth': 7, 'learning_rate': 0.045478947484788326, 'subsample': 0.9303329319971416, 'colsample_bytree': 0.5834920405813413, 'min_child_weight': 4, 'gamma': 3.3999722977718863, 'reg_alpha': 0.0256061941069305, 'reg_lambda': 4.704455246562588e-08}. Best is trial 0 with value: 70210.4968106389.
[I 2025-12-09 11:26:19,976] Trial 1 finished with value: 77569.38736267782 and parameters: {'n_estimators': 616, 'max_depth': 4, 'learning_rate': 0.25664659696147324, 'subsample': 0.7470227660880565, 'colsample_bytree': 0.9949408773116604, 'min_child_weight': 10, 'gamma': 3.0135070159573987, 'reg_alpha': 2.7138806152380623, 'reg_lambda':

Best params: {'n_estimators': 556, 'max_depth': 7, 'learning_rate': 0.022796689638976034, 'subsample': 0.527518243803757, 'colsample_bytree': 0.5077990736238005, 'min_child_weight': 1, 'gamma': 1.8807794694524913, 'reg_alpha': 0.10626430173302445, 'reg_lambda': 1.3619344128142352e-08}


In [9]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32644.66471717254
RMSE: 70306.28921127891
R²: 0.9618013175165535


2025/12/09 16:48:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
